# Unit 5 -- Bayesian Modeling

|        Time | Subject                                                      |
|:------------|--------------------------------------------------------------|
| 08:30-08:45 | Unit 1 -- Welcome and Intro                                  |
| 08:45-09:30 | Unit 2 -- Financial Ecosystem: `quantmod`, `xts`, Plotting   |
| 09:45-10:30 | Unit 3 -- Volatility Modeling: `rugarch` and `rmgarch`       |
| 10:45-11:30 | Unit 4 -- Performance and Portfolio Analysis                 |
| 11:45-12:30 | **Unit 5 -- Bayesian Modeling: `Stan`**                      |


Much more detail at <http://tharte.github.io/mbt>

## Bayesian Analysis

> Statistics is the science of learning from data, and of
  measuring, controlling, and communicating uncertainty.
>  
> -- M.Davidian and T.A.Louis. *Why statistics?* Science, 336(6077):12, 2012.
  

Bayesian Statistics emphasizes the use of *probability* as a language for describing uncertainty:

![](distr.png)

High level: Achieve optimal inference by combining: 
- Prior information (expressed probabilistically)
- Observations (expressed via  data model)

Not all roses -- computation is difficult...

## `Stan`

![](https://raw.githubusercontent.com/stan-dev/logos/master/logo_tm.png)

A probabilistic programming language for performing Bayesian inference

Similar to **BUGS** and **JAGS**

Structured around "blocks." Big three: 
- **data**: what I've seen
- **parameters**: what I want to know
- **model**: how those connect

A simple example: 

- Coin with probability $p$ of coming up heads
- $N$ flips: $y_i$ outcomes
- What is our best guess $p$? 

Analytically: 
- Put a uniform prior (*a.k.a* a Beta(1, 1) distribution) on $p$
- Likelihood (data model): binomial with $Y = \sum y_i$ successes from $N$ trials

Bayes rule: 

$$\pi(p | Y) \propto \pi(p) * \pi(Y | p) = 1 * \binom{N}{Y} p^Y(1-p)^{N-y} \implies p|Y \sim \text{Beta}(Y + 1, N-Y + 1)$$

Stan Code: 

In [ ]:
file.show("bb.stan")

In [ ]:
library(rstan)
bb_model <- stan_model("bb.stan")
N <- 20
p <- 0.3
y <- rbinom(N, size=1, prob=p)

# More than enough samples for this simple model
bb_samples <- sampling(bb_model, data=list(N=N, y=y), chains=1, iter=1000)

In [ ]:
bb_samples

In [ ]:
as.matrix(bb_samples)

In [ ]:
traceplot(bb_samples)

In [ ]:
plot(density(as.matrix(bb_samples, par="p")))

In [ ]:
## Analytical posterior
analytical_samples <- rbeta(10000, 1 + sum(y), 1 + N - sum(y))
plot(density(analytical_samples))

### Stan for Regression

- Use `rstanarm` or `brms`

### Stochastic volatility model: 

$$ \varsigma_t^2 = \phi (\mu - \varsigma^2_{t-1}) + \mu $$

$$ y_t = N(0, e^{\varsigma^2}) $$

In [ ]:
file.show("sv.stan")

In [ ]:
library(quantmod)
library(rstan)

SPY <- getSymbols("SPY", auto.assign=FALSE, from="2015-01-01")
SPY.R <- na.omit(ROC(Ad(SPY)))

svmodel <- stan_model("sv.stan")
svsamples <- sampling(svmodel, data=list(y=as.vector(SPY.R), T = length(SPY.R)), 
                      chains=1, iter=1000)

In [ ]:
plot(abs(as.vector(SPY.R)), type="l")
lines(colMeans(as.matrix(svsamples, pars="sigmaT")), col="red4", lwd=3)

The usefulness of Stan is not in its ability to fit *simple* models, but in it's ability to fit complex models flexibly

Switching to asymmetric GARCH, different return distributions, *etc.* are small (one or two line) changes

*E.g.*, using a Student-$t$ distribution with unknown DF for the returns: 

```
parameters{
...
real<lower=0> nu;
...
}

model{
...
nu ~ cauchy(0, 5);
y ~ student_t(nu, 0, exp(h/2));
...
}
```

Or a skew normal:

```
parmeters{
    ...
    real alpha;
    ...
}
model{
    ...
    alpha ~ cauchy(0, 5);
    y ~ skew_normal(0, exp(h/2), alpha);
    ...
}
```

Folk theorem: Difficult MCMC is a good proxy for model misspecification 

(formalizing this is hard...)

### GARCH Model

From the Stanual: <http://mc-stan.org/users/documentation/>

In [ ]:
file.show("garch.stan")

In [ ]:
garch_model <- stan_model("garch.stan")
garch_samples <- sampling(garch_model, data=list(y=as.vector(SPY.R), T = length(SPY.R), sigma1 = mean(abs(SPY.R))), 
                          chains=1, iter=1000)

In [ ]:
garch_samples

In [ ]:
plot(abs(as.vector(SPY.R)), type="l")
lines(colMeans(as.matrix(garch_samples, pars="sigma")), col="red4", lwd=3)

### HMM-GARCH

From Damiano, Peterson, and W: <https://rawgit.com/luisdamiano/stancon18/master/main.html#a-markov-switching-garch-model>

In [ ]:
file.show("hmm_garch.stan")

In [ ]:
hmmgarch_model <- stan_model("hmm_garch.stan")
hmmgarch_samples <- sampling(hmmgarch_model, data=list(y=as.vector(SPY.R), T = length(SPY.R)), 
                             chains=1, iter=1000)

In [ ]:
hmmgarch_samples

In [ ]:
plot(abs(as.vector(SPY.R)), type="l")
lines(colMeans(as.matrix(hmmgarch_samples, pars="sigma_t")), col="red4", lwd=3)

In [ ]:
extract(hmmgarch_samples, pars="sigma_t")